## Analysis of EV Charging Station Availability for a Roadtrip Through the Midwest USA

##### Electric vehicles are starting to penetrate the consumer auto market. One significant barrier to their adoption by consumers is the availability of charging stations for non-local excursions. Electric cars have ranges of 300-500 miles, so having confidence in being able to find a charging port while planning any significant road trip becomes rather difficult. Obtaining insight into the availability of charging ports at major cities - and their proximity to hotels - would provide a major boost of confidence to the road trip consumer who owns an electric vehicle. This data could also be used to help determine where a new charging station may be best suited to maximize availability for consumers.

##### FourSquare includes in its a database a list of EV charging locations. To help build confidence in a road trip plan, this data can be used and combined with FourSquare's city hotel data to help plan an itinerary that includes "distance to charging port" as a main criterion, along with typical decisional factors like hotel rating and location. For each major city on the route, the location of hotels can be visualized by using clustering to find highly rated hotels which have a charging station within walking distance. Performing this exercise at each major city stopping point along the way will yield a road trip itinerary that ensures adequate and convenient charging capability for an electric vehicle.



In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!pip install folium
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [2]:
CLIENT_ID = 'UJV5PXV1XQOWNEWWMKLKAV52BC0NUD3NLCSMH013P112YYPR' # your Foursquare ID
CLIENT_SECRET = 'EIU5V1AYEG032Y0V03DEHRHVNHG2DVG0F1XPP022SPLXVM0T' # your Foursquare Secret
VERSION = '20200720' # Foursquare API version
LIMIT = 100

In [3]:
address = ['Los Angeles, CA', 'Las Vegas, NV', 'Provo, UT', 'Denver, CO', 'Topeka, KS', 'St. Louis, MO', 'Cincinnati, OH']
latitude_list = []
longitude_list = []

for add in address:
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(add)
    latitude_list.append(location.latitude)
    longitude_list.append(location.longitude)
    #print(latitude, longitude)

In [4]:
radius_hotel = 12000.0
radius_charging_limit = 1500.0

hotel_category_id = '4bf58dd8d48988d1fa931735'
charging_category_id = '5032872391d4c4b30a586d64'



In [5]:
latitude = latitude_list[0]
longitude = longitude_list[0]

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, hotel_category_id, radius_hotel, LIMIT)

In [6]:
results_hotel = requests.get(url).json()


In [17]:
# assign relevant part of JSON to venues
venues_hotel = results_hotel['response']['venues']

# tranform venues into a dataframe
dataframe_hotel = json_normalize(venues_hotel)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe_hotel.columns if col.startswith('location.')] + ['id']
dataframe_hotel_filtered = dataframe_hotel.loc[:, filtered_columns]

In [18]:


# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_hotel_filtered['categories'] = dataframe_hotel_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_hotel_filtered.columns = [column.split('.')[-1] for column in dataframe_hotel_filtered.columns]
dataframe_hotel_filtered['Rating']=np.nan
dataframe_hotel_filtered['Stations_Count_1500']=np.nan
dataframe_hotel_filtered['Stations_Count_1000']=np.nan
dataframe_hotel_filtered['Stations_Count_500']=np.nan
dataframe_hotel_filtered['Station Min Distance']=np.nan

initialize = ['a','b','c']

dataframe_hotel_filtered['Stations_Lat_1500']=str(initialize)
dataframe_hotel_filtered['Stations_Lat_1000']=str(initialize)
dataframe_hotel_filtered['Stations_Lat_500']=str(initialize)
dataframe_hotel_filtered['Stations_Lng_1500']=str(initialize)
dataframe_hotel_filtered['Stations_Lng_1000']=str(initialize)
dataframe_hotel_filtered['Stations_Lng_500']=str(initialize)


In [20]:
radius_charging_limit = 1500.0

Hotel_Lat_List = dataframe_hotel_filtered['lat'].tolist()
Hotel_Lng_List = dataframe_hotel_filtered['lng'].tolist()
Hotel_id_List = dataframe_hotel_filtered['id'].tolist()

for Latitude, Longitude, venue_id in zip(Hotel_Lat_List,Hotel_Lng_List,Hotel_id_List):
    
    url_hotel_venue = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result_hotel_venue = requests.get(url_hotel_venue).json()
    print(result_hotel_venue)
    try:
        rating = result_hotel_venue['response']['venue']['rating']
    except:
        rating = np.nan
    
    print(rating)
    dataframe_hotel_filtered.loc[dataframe_hotel_filtered.id==venue_id,'Rating'] = rating

    url_charging = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, Latitude, Longitude, VERSION, charging_category_id, radius_charging_limit, LIMIT)
    results_charging_station = requests.get(url_charging).json()

    # assign relevant part of JSON to venues
    try:
        venues_charging_station = results_charging_station['response']['venues']
        # tranform venues into a dataframe
        dataframe_charging_station = json_normalize(venues_charging_station)
    except:
        print('exception')
        dataframe_charging_station = pd.DataFrame(columns=[])
    
    print(dataframe_charging_station.size)
    if dataframe_charging_station.size > 0:
        # keep only columns that include venue name, and anything that is associated with location
        filtered_columns = ['name', 'categories'] + [col for col in dataframe_charging_station.columns if col.startswith('location.')] + ['id']
        dataframe_charging_station_filtered = dataframe_charging_station.loc[:, filtered_columns]
        
        # filter the category for each row
        dataframe_charging_station_filtered['categories'] = dataframe_charging_station_filtered.apply(get_category_type, axis=1)

        # clean column names by keeping only last term
        dataframe_charging_station_filtered.columns = [column.split('.')[-1] for column in dataframe_charging_station_filtered.columns]
        
        dataframe_charging_station_filtered_1500 = dataframe_charging_station_filtered[dataframe_charging_station_filtered['distance'] <= 1500]
        dataframe_charging_station_filtered_1000 = dataframe_charging_station_filtered[dataframe_charging_station_filtered['distance'] <= 1000]
        dataframe_charging_station_filtered_500 = dataframe_charging_station_filtered[dataframe_charging_station_filtered['distance'] <= 500]
        
        count_1500 = dataframe_charging_station_filtered_1500.id.count()
        count_1000 = dataframe_charging_station_filtered_1000.id.count()
        count_500 = dataframe_charging_station_filtered_500.id.count()

        lat_1500 = dataframe_charging_station_filtered_1500['lat'].tolist()
        lng_1500 = dataframe_charging_station_filtered_1500['lng'].tolist()

        lat_1000 = dataframe_charging_station_filtered_1000['lat'].tolist()
        lng_1000 = dataframe_charging_station_filtered_1000['lng'].tolist()

        lat_500 = dataframe_charging_station_filtered_500['lat'].tolist()
        lng_500 = dataframe_charging_station_filtered_500['lng'].tolist()
        
        dataframe_hotel_filtered.loc[dataframe_hotel_filtered.id==venue_id,'Stations_Count_1500'] = count_1500
        dataframe_hotel_filtered.loc[dataframe_hotel_filtered.id==venue_id,'Stations_Count_1000'] = count_1000
        dataframe_hotel_filtered.loc[dataframe_hotel_filtered.id==venue_id,'Stations_Count_500'] = count_500

        dataframe_hotel_filtered.loc[dataframe_hotel_filtered.id==venue_id,'Stations_Lat_1500'] = str(lat_1500)
        dataframe_hotel_filtered.loc[dataframe_hotel_filtered.id==venue_id,'Stations_Lat_1000'] = str(lat_1000)
        dataframe_hotel_filtered.loc[dataframe_hotel_filtered.id==venue_id,'Stations_Lat_500'] = str(lat_500)

        dataframe_hotel_filtered.loc[dataframe_hotel_filtered.id==venue_id,'Stations_Lng_1500'] = str(lng_1500)
        dataframe_hotel_filtered.loc[dataframe_hotel_filtered.id==venue_id,'Stations_Lng_1000'] = str(lng_1000)
        dataframe_hotel_filtered.loc[dataframe_hotel_filtered.id==venue_id,'Stations_Lng_500'] = str(lng_500)    
        
        dataframe_hotel_filtered.loc[dataframe_hotel_filtered.id==venue_id,'Station Min Distance'] = min(dataframe_charging_station_filtered['distance'])
        

{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f1b37faeb4b917f81e2f792'}, 'response': {}}
nan
0
{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f1b36b0241e64486cfccd70'}, 'response': {}}
nan
32
{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f1b366e5dc8d450a1d9e2d9'}, 'response': {}}
nan
0
{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f1b3740979acd6d9119c6d1'}, 'response': {}}
nan
0
{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f1b35885292fd2d2fc593cf'}, 'response': {}}
nan
32
{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5f1b37751e406e2232357aa0'}, 'response': {}}
nan
17
{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'req

In [10]:
dataframe_hotel_filtered.dropna(subset=['Rating','Stations_Count_1500'],inplace=True)
dataframe_hotel_filtered.drop(columns=['country','cc','categories','crossStreet','formattedAddress','labeledLatLngs','neighborhood','postalCode','state'],inplace=True)
#dataframe_hotel_filtered = dataframe_hotel_filtered[dataframe_hotel_filtered.Stations_Count_1500 > 0]

In [11]:
dataframe_hotel_filtered.head()

,name,address,city,distance,lat,lng,id,Rating,Stations_Count_1500,Stations_Count_1000,Stations_Count_500,Station Min Distance,Stations_Lat_1500,Stations_Lat_1000,Stations_Lat_500,Stations_Lng_1500,Stations_Lng_1000,Stations_Lng_500


In [12]:
dataframe_hotel_filtered['Rating']

Series([], Name: Rating, dtype: float64)

In [13]:
dfObj = pd.DataFrame(columns=[])